#To be able to see the plotly figures, use:
https://nbviewer.org/github/damienld/TennisShots/blob/main/shot_analysis.ipynb

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import warnings
from plotly import subplots
import plotly.express as px
warnings.filterwarnings('ignore')


In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

filename_csv="Pospisil Wolf.csv"
df=pd.read_csv(filename_csv, index_col=0)
df["Winner_Point"]=df["Winner_Point"].astype(str)
lst_players=filename_csv.split(".")[0].split(" ")
name1=lst_players[0]
name2=lst_players[1]
current_point=0
player=None #videplayer

def transform_far_to_near(df):
    if len(df) == 0:
        return df
    df["Bounce (x)"] = df.apply(lambda x: -(x["Bounce (x)"]), axis=1)
    df["Bounce (y)"] = df.apply(lambda x: 23.73 - (x["Bounce (y)"]), axis=1)
    return df


def trace_rect(left_double_court_x, right_double_court_x, top_double_court_y):
    """
    outside lines
    """
    court_x = [
        left_double_court_x,
        left_double_court_x,
        right_double_court_x,
        right_double_court_x,
    ]
    court_y = [0, top_double_court_y, top_double_court_y, 0]
    trace0 = go.Scatter(x=court_x, y=court_y, mode="lines", line=dict(color="white"))
    # fig.add_trace(trace0)

    # trace0=go.Scatter(
    return go.Scatter(x=court_x, y=court_y, mode="lines", line=dict(color="white"))
    # fig.add_trace(trace0)


def trace_court(xrange, yrange):
    # fig1 = px.line()
    fig1 = go.Figure(
        layout_xaxis_range=xrange,
        layout_yaxis_range=yrange,
    )
    fig1.add_trace(trace_rect(0 - 4.11 - 1.37, 0 + 4.11 + 1.37, 23.73))
    fig1.add_trace(trace_rect(0 - 4.11, 0 + 4.11, 23.73))  # top baseline line
    fig1.add_trace(trace_rect(0 - 4.11, 0 + 4.11, 11.88))  # net1
    fig1.add_trace(trace_rect(0 - 4.11, 0 + 4.11, 5.48))  # bottom service line
    fig1.add_trace(trace_rect(0 - 4.11, 0 + 4.11, 12.08))  # net2
    fig1.add_trace(trace_rect(0 - 4.11, 0 + 4.11, 18.48))  # top service line"""
    fig1.update_layout(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
    fig1.update_layout(xaxis=dict(zeroline=False), yaxis=dict(zeroline=False))
    return fig1

def get_all_serves_player(df, isP1: bool, index1stServe: int):
    """Returns all serves for a given player
    Turns all far serves coordinates into near serves coordinates
    Args:
        df (_type_): _description_
        isP1 (bool): _description_
        index1stServe (int): 0= All, 1=1st, 2=2nd
    Returns:
        _type_: Dataframe with all serves
    """
    player_str = "Player" if isP1 else "Opponent"
    dfServesIn_near = df[
        (df.Shot == 1)
        & (df.Player == player_str)
        & (df.Result == "In")
        & (df["Hit Side"] == "near")
    ]
    dfServesIn_far = df[
        (df.Shot == 1)
        & (df.Player == player_str)
        & (df.Result == "In")
        & (df["Hit Side"] == "far")
    ]

    df_near_deuce = dfServesIn_near[dfServesIn_near["Hit Zone"] == "deuce"]
    df_far_deuce = dfServesIn_far[dfServesIn_far["Hit Zone"] == "deuce"]
    df_far_deuce = transform_far_to_near(df_far_deuce)

    df_near_ad = dfServesIn_near[dfServesIn_near["Hit Zone"] == "ad"]
    df_far_ad = dfServesIn_far[dfServesIn_far["Hit Zone"] == "ad"]
    df_far_ad = transform_far_to_near(df_far_ad)

    df_all = pd.concat([df_near_deuce, df_far_deuce, df_near_ad, df_far_ad])
    if index1stServe == 1:
        df_all = df_all[df_all["first_serve"]==True]
    elif index1stServe == 2:
        df_all = df_all[df_all["first_serve"]==False]
    #df_all["Winner_Point"] = df_all["Winner_Point"].replace({"1": "won", "2": "lost"})
    return df_all

isP1=True
#get all 1st serves
print("1st serves:"+ str(len(get_all_serves_player(df, isP1, 1))))
#get all 2nd serves
print("2nd serves:"+ str(len(get_all_serves_player(df, isP1, 2))))
#get all serves
print("All serves:"+ str(len(get_all_serves_player(df, isP1, 0))))

1st serves:51
2nd serves:36
All serves:87


In [4]:

def trace_point(df, name1, name2, index_point, shot=None, x_new=None, y_new=None):
    # print("trace_point " + str(current_point))
    fig1 = trace_court([-7, 7], [-5, 30])
    df_temp = df
    # trace all net shots to end into the net
    df_temp.loc[
        (df["Result"] == "Net") & (df["Point"] == index_point), "Bounce (y)"
    ] = 11.88
    if shot is None:
        df_point = df_temp[
            (df_temp["Point"] == index_point) & (df_temp["Shot"] > 0)
        ].sort_values(by="Shot")
    else:
        df_point = df_temp[
            (df_temp["Point"] == index_point)
            & (df_temp["Shot"] <= shot + 1)
            & (df_temp["Shot"] >= shot - 1)
        ].sort_values(by="Shot")
    last_shot = None
    fig4 = None

    # "LAST SHOT"
    if len(df_point) > 0:
        # trace a line from last hit position to last bounce to display the last shot
        last_shot = df_point.iloc[-1]
        # print(last_shot["Hit (x)"])
        df2 = pd.DataFrame(
            dict(
                x=[last_shot["Hit (x)"], last_shot["Bounce (x)"]],
                y=[last_shot["Hit (y)"], last_shot["Bounce (y)"]],
            )
        )
        df2["Text"] = ""  # last_shot["Shot"]
        # df2["Text"]=["   "+ s for s in df2["Text"]]
        # df2.loc[:,"Text"]=('    ' + df['Text'])
        fig4 = px.line(df2, x="x", y="y", text="Text")
        color = "red"
        if last_shot["Result"] == "In":
            color = "green"
        fig4.update_traces(patch={"line": {"width": 2, "dash": "dot", "color": color}})

    # Figure HIT POSITION
    fig2 = px.scatter(
        df_point,
        x="Hit (x)",
        y="Hit (y)",
        text="Shot"  # , color="Player"
        # , symbol="Stroke"
        # , animation_frame="Shot"#, animation_group="Player"
        # , textposition='top right',textfont=dict(color='#E58606')
        # , symbol="Spin"
        # , trendline='ols',trendline_color_override='red'
        # , marginal_y="rug"
        # , marginal_x="histogram"
        # , facet_col="Winner_Point"#, facet_row="time"
        ,
        color_discrete_map={"Player": "green", "Opponent": "red"},
        hover_data=["Shot", "Stroke"],
    )

    fig2.for_each_trace(
        lambda t: t.update(
            mode="lines+markers+text",
            textposition="middle right",
            textfont=dict(size=10, color="gray"),
        )
    )
    for fr in fig2.frames:
        for d in fr.data:
            d.update(
                mode="markers+lines+text",
                textposition="middle right",
                textfont=dict(size=10, color="gray"),
            )
    fig2.update_traces(patch={"line": {"width": 1, "dash": "dot"}})
    """marker_symbol=[1,101], """

    # FIG BOUNCE
    fig3 = px.scatter(
        df_point,
        x="Bounce (x)",
        y="Bounce (y)",
        color="Result"
        # , trendline='ols',trendline_color_override='red'
        # , marginal_y="rug"
        # , marginal_x="histogram"
        # , facet_col="Winner_Point"#, facet_row="time"
        ,
        color_discrete_map={"Net": "black", "In": "orange", "Out": "black"},
        hover_data=["Shot"],
        text="Shot",
    )
    fig3.for_each_trace(
        lambda t: t.update(
            mode="markers+text",
            textposition="middle right",
            textfont=dict(size=9, color="gray"),
        )
    )

    # add NEW POINT
    fig5 = None
    if x_new is not None:
        df3 = pd.DataFrame(dict(x=[x_new], y=[y_new], text="NEW"))
        fig5 = px.scatter(df3, x="x", y="y", text="text")

    if fig4 is not None:
        if fig5 is not None:
            fig = go.Figure(
                data=fig1.data + fig2.data + fig3.data + fig4.data + fig5.data
            )
        else:
            fig = go.Figure(data=fig1.data + fig2.data + fig3.data + fig4.data)
    else:
        if fig5 is not None:
            fig = go.Figure(data=fig1.data + fig2.data + fig3.data + fig5.data)
        else:
            fig = go.Figure(data=fig1.data + fig2.data + fig3.data)
    # fig.update_layout(title="Serves P1 (point size represents Speed)")
    fig.update_xaxes(
        scaleanchor="y",
        scaleratio=3,
    )
    fig.update_layout(xaxis_range=[-5, 5], yaxis_range=[-4, 11.88 * 2 + 4])
    fig.update_layout(
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False),
        autosize=False,
        width=850,
        height=400,
    )
    f = go.FigureWidget(fig)

    # add ANNOTATION with name player
    if len(df_point) > 0:
        first_shot = df_point.iloc[0]
        x = 6
        if first_shot["Hit Zone"] == "deuce" and first_shot["Hit Side"] == "far":
            x = -6
        if first_shot["Hit Zone"] == "ad" and first_shot["Hit Side"] == "near":
            x = -6
        y = -1.5
        if first_shot["Hit Side"] == "far":
            y = 25.5
        name_p_temp = name1
        if first_shot["Player"] == "Opponent":
            name_p_temp = name2
        name_p_temp = name_p_temp[0:6]
        f.add_annotation(
            x=x, y=y, text=name_p_temp, showarrow=False  # arrows' head  # arrows' head
        )

    """scatter = f.data[-1]
    #print(scatter)
    colors = ['#a3a7e4'] * 100
    scatter.marker.color = colors
    scatter.marker.size = [10] * 100
    f.layout.hovermode = 'closest'


    # create our callback function
    def update_point(trace, points, selector):
        #print("click")
        c = list(scatter.marker.color)
        s = list(scatter.marker.size)
        for i in points.point_inds:
            c[i] = '#bae2be'
            s[i] = 50
            with f.batch_update():
                scatter.marker.color = c
                scatter.marker.size = s


    scatter.on_click(update_point)"""

    f

    f.show()


def trace_serve_3areas():
    a=0.35
    b=0.3
    c=0.35
    fig2_1 = px.line(x=[-4.11 * (a+b), -4.11 * (a+b)], y=[18.48, 12.5])
    fig2_2 = px.line(x=[-4.11 * (c), -4.11 * (c)], y=[18.48, 12.5])
    fig2_3 = px.line(x=[4.11 * (a+b), 4.11 * (a+b)], y=[18.48, 12.5])
    fig2_4 = px.line(x=[4.11 * (c), 4.11 * (c)], y=[18.48, 12.5])
    fig = go.Figure(data=fig2_1.data + fig2_2.data + fig2_3.data + fig2_4.data)
    fig.update_traces(patch={"line": {"color": "gray", "width": 1, "dash": "dot"}})
    return fig


def trace_court_and_serves(df, isP1: bool, index1stServe: int, mytitle: str):
    fig1 = trace_court([-5, 5], [-0.5, 24])
    fig2 = trace_serve_3areas()
    df_serves = get_all_serves_player(df, isP1, index1stServe)
    fig3 = px.scatter(
        df_serves,
        x="Bounce (x)",
        y="Bounce (y)",
        # size=((df_serves["Speed (KM/H)"].clip(120, 180)) - 119) / 100,
        # size_max=10,

        # , trendline='ols',trendline_color_override='red'
        # , marginal_y="rug"
        # , marginal_x="histogram"
        # , facet_col="Winner_Point"#, facet_row="time"
        color="Winner_Point",
        color_discrete_map={"1": "green", "2": "red"},
        hover_data=["Point","Speed (KM/H)"],
    )
    fig3.update_traces(showlegend=False)
    #fig3.show()
    fig = go.Figure(data=fig1.data + fig2.data + fig3.data)

    # fig.update_layout(title="Serves P1 (point size represents Speed)")
    fig.update_xaxes(
        scaleanchor="y",
        scaleratio=1.5,
    )
    # fig.update_layout(xaxis_range=[-5, 5], yaxis_range=[-2, 11.88 * 2 + 1])
    fig.update_layout(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False),title=mytitle)
    fig.update(layout_yaxis_range=[-0.4, 24])
    fig.update(layout_xaxis_range=[-5, 5])
    fig.update_traces(showlegend=False)
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    return fig  # .show()

index1stServe=1
fig1_1=trace_court_and_serves(df, True, index1stServe,name1+" 1st serves hitting zones and points won/lost")
fig1_1.show()
index1stServe=2
fig1_2=trace_court_and_serves(df, True, index1stServe,name1+" 2nd serves hitting zones and points won/lost")
fig1_2.show()

In [5]:
# This is a cell to hide code snippets from displaying
# This must be at first cell!

from IPython.display import HTML

hide_me = ''
HTML('''<script>
code_show=true; 
function code_toggle() {
  if (code_show) {
    $('div.input').each(function(id) {
      el = $(this).find('.cm-variable:first');
      if (id == 0 || el.text() == 'hide_me') {
        $(this).show();
      }
    });
    $('div.output_prompt').css('opacity', 1);
  } else {
    $('div.input').each(function(id) {
      $(this).show();
    });
    $('div.output_prompt').css('opacity', 1);
  }
  code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input style="opacity:0" type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [6]:
#REMOVE ALL SHOTS where shot=0 

print("df before:{0}".format(len(df)))
df=df[df["Shot"]>0]
print("df after:{0}".format(len(df)))
#df.rename(columns = {'Speed (KM/H)':'Speed'," Hit (x)":"hit_x"," Hit (y)":"hit_y"," Hit (z)":"hit_z"}, inplace = True)
dfP1=df[df.Player=="Player"]
print("dfP1:{0}".format(len(dfP1)))
dfP2=df[df.Player=="Opponent"]
print("dfP2:{0}".format(len(dfP2)))

df before:954
df after:796
dfP1:404
dfP2:392


# Scatter

In [7]:
hide_me

#créer une figure
fig = go.Figure()
#créer une courbe dont les x sont « Stroke » et les y sont « Speed »
trace1 = go.Scatter(
    x=dfP1.Stroke, y=dfP1["Speed (KM/H)"],
    text=dfP1.Player, #annote les points avec cette variable
    mode="markers", #mode=line+markers
    marker=dict(color='blue'), name=name1
)
fig.add_trace(trace1)
#ajouter une autre courbe
trace2 = go.Scatter(
    x=dfP2.Stroke, y=dfP2["Speed (KM/H)"],
    text=dfP2.Player, #annote les points avec cette variable
    mode="markers", #mode=line+markers
    marker=dict(color='red'), name=name2
)
fig.add_trace(trace2)
#layout
fig.update_layout(title="Shot Speed vs Shot type", xaxis_title="Shot type", yaxis_title="Shot Speed")
#afficher
fig.show()

# Bar

In [8]:
hide_me
#créer une figure

trace3 = go.Bar(
    x=dfP1.Stroke, y=dfP1["Speed (KM/H)"],
    text=dfP1.Player, #annote les points avec cette variable
    marker=dict(color='blue'), name=name1
)
trace4 = go.Bar(
    x=dfP2.Stroke, y=dfP2["Speed (KM/H)"],
    text=dfP2.Player, #annote les points avec cette variable
    marker=dict(color='red'), name=name2
)
data2=[trace3, trace4]
#layout
layout2=go.Layout(barmode="group", title="Cumulated Shot Speed by Stroke"
                  , xaxis_title="Stroke", yaxis_title="Shot Speed")
fig2=go.Figure(data=data2, layout=layout2)
#afficher
fig2.show()

# Multiple Subplots

In [9]:
hide_me
#créer une courbe dont les x sont « world_rank » et les y sont « citations »
trace1 = go.Scatter(
    x=dfP1.Shot, y=dfP1["Speed (KM/H)"],
    text=dfP1.Player, #annote les points avec cette variable
    mode="markers", #mode=line+markers
    marker=dict(color='blue'), name=name1
)
fig.add_trace(trace1)
#ajouter une autre courbe
trace2 = go.Scatter(
    x=dfP2.Shot, y=dfP2["Speed (KM/H)"],
    text=dfP2.Player, #annote les points avec cette variable
    mode="markers", #mode=line+markers
    marker=dict(color='red'), name=name2
)
#create 4 subplots into the figure
fig3= subplots.make_subplots(rows=2, cols=2, shared_xaxes=True
                             , shared_yaxes=True, vertical_spacing=0.1
                            , horizontal_spacing=0.1)
#add trace on figure
fig3.append_trace(trace3, 1, 1)
fig3.append_trace(trace1, 1, 2)
fig3.append_trace(trace4, 2, 1)
fig3.append_trace(trace2, 2, 2)
#afficher
fig3.show()

# Box Plot

In [10]:
hide_me
#créer une figure
fig = go.Figure()
#créer une courbe dont les x sont « world_rank » et les y sont « citations »
trace1 = go.Box(
    x=dfP1.Stroke, y=dfP1["Speed (KM/H)"],
    text=dfP1.Player, #annote les points avec cette variable
    marker=dict(color='blue'), name=name1
)
fig.add_trace(trace1)
#ajouter une autre courbe
trace2 = go.Box(
    x=dfP2.Stroke, y=dfP2["Speed (KM/H)"],
    text=dfP2.Player, #annote les points avec cette variable
    marker=dict(color='red'), name=name2
)
fig.add_trace(trace2)
#layout
fig.update_layout(title="Shot Speed vs Shot type", xaxis_title="Shot type", yaxis_title="Shot Speed")
#afficher
fig.show()

# Pie Charts

In [11]:
def pie_chart(labels, lst, nameP):
    trace5 = go.Pie(labels=labels, values=lst)
    layout5= go.Layout(title="Spins for " + nameP)

    fig4= go.Figure(data=trace5, layout=layout5)
    fig4.show()

In [12]:

hide_me
lst =dfP1["Spin"].value_counts()
labels=(lst.index)
print(lst)
pie_chart(labels, lst, name1)

Topspin    167
Flat       142
Slice       82
Kick         3
Name: Spin, dtype: int64


In [13]:
hide_me
lst =dfP2["Spin"].value_counts()
print(lst)
labels=(lst.index)
labels
pie_chart(labels, lst, name2)


Flat       164
Topspin    150
Slice       56
Kick        15
Name: Spin, dtype: int64


# SERVES

In [14]:
dfServesInP1_near=df[(df.Stroke=="Serve") & (df.Player=="Player")& (df.Result=="In")& (df["Hit Side"]=="near")]
print("dfServesInP1_near:{0}".format(len(dfServesInP1_near)))
dfServesInP1_far=df[(df.Stroke=="Serve") & (df.Player=="Player")& (df.Result=="In")& (df["Hit Side"]=="far")]
print("dfServesInP1_far:{0}".format(len(dfServesInP1_far)))
dfServesInP2_near=df[(df.Stroke=="Serve") & (df.Player!="Player")& (df.Result=="In")& (df["Hit Side"]=="near")]
print("dfServesInP2_near:{0}".format(len(dfServesInP2_near)))
dfServesInP2_far=df[(df.Stroke=="Serve") & (df.Player!="Player")& (df.Result=="In")& (df["Hit Side"]=="far")]
print("dfServesInP2_far:{0}".format(len(dfServesInP2_far)))

dfServesInP1_near:38
dfServesInP1_far:49
dfServesInP2_near:56
dfServesInP2_far:39


In [15]:
hide_me
#créer une figure
#fig = go.Figure()
def transform_far_to_near(df):
    df["Bounce (x)"]=df.apply(lambda x: -(x["Bounce (x)"]), axis=1)
    df["Bounce (y)"]=df.apply(lambda x: 23.73-(x["Bounce (y)"]), axis=1)
    return df
                              
def trace_rect(fig1, left_double_court_x,right_double_court_x, top_double_court_y ):
    """
    outside lines
    """
    court_x=[left_double_court_x, left_double_court_x, right_double_court_x, right_double_court_x]
    court_y=[0,top_double_court_y, top_double_court_y, 0]
    trace0=go.Scatter(
        x=court_x, y=court_y,
        mode="lines",line=dict(color='white')
    )
    #fig.add_trace(trace0)
    
    #trace0=go.Scatter(
    fig1.add_trace(go.Scatter(
        x=court_x, y=court_y,
        mode="lines",line=dict(color='white')
    ))
    #fig.add_trace(trace0)

def trace_court():
    fig1 = px.line()
    trace_rect(fig1, 0-4.11-1.37, 0+4.11+1.37, 23.73)
    trace_rect(fig1, 0-4.11, 0+4.11, 23.73)
    trace_rect(fig1, 0-4.11, 0+4.11, 11.88)
    trace_rect(fig1, 0-4.11, 0+4.11, 5.48)
    trace_rect(fig1, 0-4.11, 0+4.11, 12.08)
    trace_rect(fig1, 0-4.11, 0+4.11, 18.48)
    return fig1

    
fig1 = trace_court()
fig2 = trace_serve_3areas()
fig2.update_traces(patch={"line": {"color": "gray", "width": 1, "dash": 'dot'}}) 

df_to_check_1=dfServesInP2_near[dfServesInP2_near["Hit Zone"]=="deuce"]
df_to_check_2=dfServesInP2_far[dfServesInP2_far["Hit Zone"]=="deuce"]
df_to_check_2=transform_far_to_near(df_to_check_2)

df_to_check_3=dfServesInP2_near[dfServesInP2_near["Hit Zone"]=="ad"]
df_to_check_4=dfServesInP2_far[dfServesInP2_far["Hit Zone"]=="ad"]
df_to_check_4=transform_far_to_near(df_to_check_4)

df_to_check = pd.concat([df_to_check_1, df_to_check_2, df_to_check_3, df_to_check_4])
df_to_check["Winner_Point"]=df_to_check["Winner_Point"].replace({"1":"won","2":"lost"})
fig3=px.scatter(df_to_check,x="Bounce (x)",y="Bounce (y)"
                #,size=df_to_check["Speed (KM/H)"].clip(170, 220)-150, size_max=10
                #, symbol="Spin"
                   #, trendline='ols',trendline_color_override='red'
                    #, marginal_y="rug"
                    #, marginal_x="histogram"
                    #, facet_col="Winner_Point"#, facet_row="time"
                    ,color="Winner_Point", color_discrete_map={"won":"green","lost":"red"}
                    , hover_data=["Point","Speed (KM/H)"]
                   )

fig=go.Figure(data=fig1.data + fig2.data + fig3.data)

fig.update_layout(title="Serves "+name2+" (point size represents Speed)")
fig.update_xaxes(
    scaleanchor = "y",
    scaleratio = 2,
  )
fig.update_layout(xaxis_range=[-5,5], yaxis_range=[-2,11.88*2+1])
fig.update_layout(
     xaxis=dict(showgrid=False), 
     yaxis=dict(showgrid=False)
)
fig.show()


In [16]:
hide_me
fig4=px.histogram(df_to_check[df_to_check["Hit Zone"]=="deuce"],x="Bounce (x)",color="Winner_Point"
                    , color_discrete_map={"won":"green","lost":"red"}
                    , barmode="group", width=500
                    , hover_data=["Speed (KM/H)"], nbins=3
                    , title="Serves (to deuce side)"
                  
                   )
bin_size=4.11/3
fig4.update_traces(xbins=dict( # bins used for histogram
        start=-4.11,
        end=0,
        size=bin_size
    ))
fig4.update_layout(
    xaxis_title="Areas: Outside / Center / T",
    yaxis_title="Point Results",
)
#newnames = {'-3.43':'hello', '-2/06': 'hi'}
#fig4.for_each_trace(lambda t: t.update(name = newnames[t.name]))
fig4.show()


fig5=px.histogram(df_to_check[df_to_check["Hit Zone"]=="ad"],x="Bounce (x)",color="Winner_Point"
                    , color_discrete_map={"won":"green","lost":"red"}
                    , barmode="group", width=500
                    , hover_data=["Speed (KM/H)"], nbins=3
                    , title="Serves (to Ad side)"
                   )
fig5.update_traces(xbins=dict( # bins used for histogram
        start=0,
        end=4.11,
        size=bin_size
    ))
fig5.update_layout(
    xaxis_title="Areas: T / Center / Outside",
    yaxis_title="Point Results",
)
fig5.show()

In [17]:
hide_me
df_serves = df_to_check[["Direction","Hit Zone","Winner_Point","Spin"]]#[df_to_check["Hit Zone"]=="deuce"]
pivot=pd.pivot_table(df_serves, index=["Hit Zone"], columns=["Winner_Point"],aggfunc='count')
#df_serves_deuce.groupby("Winner_Point")["Winner_Point","Direction"].count()
print(pivot.head())
lost=round(pivot["Spin"]["lost"]*100/(pivot["Spin"]["lost"]+pivot["Spin"]["won"]))
#print(lost)
won=round(pivot["Spin"]["won"]*100/(pivot["Spin"]["lost"]+pivot["Spin"]["won"]))
#print(won)
df_serves=pd.concat([won, lost], axis=1)
df_serves.columns=["won","lost"]
df_serves.head()

             Direction     Spin    
Winner_Point      lost won lost won
Hit Zone                           
ad                  28  16   28  16
deuce               30  19   30  19


,won,lost
Hit Zone,,
ad,36.0,64.0
deuce,39.0,61.0


In [18]:
hide_me
df_serves = df_to_check[["Direction","Hit Zone","Winner_Point","Spin"]]#[df_to_check["Hit Zone"]=="deuce"]
pivot=pd.pivot_table(df_serves, index=["Hit Zone","Direction"], columns=["Winner_Point"],aggfunc='count')
#df_serves_deuce.groupby("Winner_Point")["Winner_Point","Direction"].count()
lost=round(pivot["Spin"]["lost"]*100/(pivot["Spin"]["lost"]+pivot["Spin"]["won"]))
#print(lost)
won=round(pivot["Spin"]["won"]*100/(pivot["Spin"]["lost"]+pivot["Spin"]["won"]))
#print(won)
df_serves=pd.concat([won, lost], axis=1)
df_serves.columns=["won","lost"]
df_serves.head()

won  lost
Hit Zone Direction             
ad       down the T  38.0  62.0
         out wide    36.0  64.0
deuce    down the T  42.0  58.0
         out wide    31.0  69.0

In [19]:

def transform_hit_far_to_near(df):
    df["Hit (x)"]=df.apply(lambda x: -(x["Hit (x)"]), axis=1)
    df["Hit (y)"]=df.apply(lambda x: 23.73-(x["Hit (y)"]), axis=1)
    return df

def trace_hitting_pos(player, name, is_on_serve, shot_ix):
    serve_index=is_on_serve
    serve_str="On Serve" if is_on_serve else "On Return"
    dfPlayerHits_near=df[(df["Player"]==player) & (df["Hit Side"]=="near") & (df["Shot"]%2==serve_index)]
    dfPlayerHits_far=df[(df["Player"]==player) & (df["Hit Side"]=="far") & (df["Shot"]%2==serve_index)]
    dfPlayerHits_far = transform_hit_far_to_near(dfPlayerHits_far)
    dfPlayerHits = pd.concat([dfPlayerHits_near, dfPlayerHits_far], axis=0)
    dfPlayerHits=dfPlayerHits[dfPlayerHits["Shot"]==shot_ix]
    fig3=px.scatter(dfPlayerHits,x="Hit (x)",y="Hit (y)",color="Winner_Point"
                    , color_discrete_map={"won":"green","lost":"black"}
                    , hover_data=["Speed (KM/H)","Winner_Point"]
                       )


    fig1 = trace_court()
    fig2 = px.density_contour(dfPlayerHits, x="Hit (x)", y="Hit (y)", 
                             nbinsx=6, nbinsy=6)

    fig=go.Figure(data=fig1.data + fig2.data + fig3.data)

    fig.update_layout(title="Hitting positions (2nd shot) for " + name + " " + serve_str)
    fig.update_xaxes(
        scaleanchor = "y",
        scaleratio = 2,
      )
    fig.update_layout(xaxis_range=[-5,5], yaxis_range=[-6,11.88*2-7])
    fig.update_layout(
         xaxis=dict(showgrid=False), 
         yaxis=dict(showgrid=False)
    )
    fig.show()
    
trace_hitting_pos("Opponent", name1, True,3)

In [20]:
trace_hitting_pos("Player", name1, False,4)